<a href="https://colab.research.google.com/github/sacktock/Visual-Attention-LSTM/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LSTM model for the EDVAM data set**

EDVAM: the first 3D eye tracking dataset for a virtual world.
https://github.com/YunzhanZHOU/EDVAM



**Mount Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Install dependencies**

In [3]:
!pip install livelossplot

**Imports**

In [4]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from livelossplot import PlotLosses

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Hyper-params**


In [5]:
batch_size = 128
learning_rate = 1e-4
n_epoch = 40
start_epoch = 0

**Import Dataset**

In [6]:
# Fetch the training data and labels
train_data = np.load('drive/My Drive/EDVAM_practical_subset/train.npy')
train_label = np.load('drive/My Drive/EDVAM_practical_subset/train_label.npy')

# Fetch the test data and labels
test_data = np.load('drive/My Drive/EDVAM_practical_subset/test.npy')
test_label = np.load('drive/My Drive/EDVAM_practical_subset/test_label.npy')

# Wrapper for the datasets
class EyeTrackingDataset(Dataset):
    
    def __init__(self, features, labels):
        self.features = [torch.tensor(data_item) for data_item in features]
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self,idx):
        sample = (self.features[idx], self.labels[idx])
        return sample

# Init the train set and test set
train_set = EyeTrackingDataset(train_data, train_label)
test_set = EyeTrackingDataset(test_data, test_label)

# Init the train loader and test loader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

test_iter = iter(test_set)

**Looking at the Dataset**

In [11]:
# Visualizing the data
print('Train set size:', len(train_set))
print('Test set size:',len(test_set))
print()

print('Data item:')
print(train_set[1])
print()

print('Time window:',train_set[1][0])
print()
print('Label:',train_set[1][0])
print()

print('Time window shape', test_set[1][0].shape)
print()

print('Frame:', train_set[1][0][1])
print()
print('Frame shape', train_set[1][0][1].shape)

Train set size: 190830
Test set size: 32530

Data item:
(tensor([[-1.5297e+01,  8.9990e-01, -3.8008e+00,  ...,  1.0312e+02,
          5.9100e+02,  5.9100e+02],
        [-1.5500e+01,  1.9004e+00, -4.8984e+00,  ...,  1.0312e+02,
          5.9100e+02,  5.9100e+02],
        [-1.5500e+01,  2.1992e+00, -5.3008e+00,  ...,  1.0312e+02,
          5.9100e+02,  5.9100e+02],
        ...,
        [-1.5500e+01,  2.1992e+00, -4.1016e+00,  ..., -3.9562e+01,
          1.3670e+03,  1.3670e+03],
        [-1.5500e+01,  2.1992e+00, -4.1016e+00,  ..., -3.9562e+01,
          1.3670e+03,  1.3670e+03],
        [-1.5500e+01,  2.1992e+00, -4.1016e+00,  ..., -3.9562e+01,
          1.3670e+03,  1.3670e+03]], dtype=torch.float16), 0.0)

Time window: tensor([[-1.5297e+01,  8.9990e-01, -3.8008e+00,  ...,  1.0312e+02,
          5.9100e+02,  5.9100e+02],
        [-1.5500e+01,  1.9004e+00, -4.8984e+00,  ...,  1.0312e+02,
          5.9100e+02,  5.9100e+02],
        [-1.5500e+01,  2.1992e+00, -5.3008e+00,  ...,  1.0312e+0